## We want to see if a user make a new service subscription transaction within 30 days after their current membership expiration date.

The churn/renewal definition can be tricky due to KKBox's subscription model. Since the majority of KKBox's subscription length is 30 days, a lot of users re-subscribe every month. The key fields to determine churn/renewal are transaction date, membership expiration date, and is_cancel. Note that the is_cancel field indicates whether a user actively cancels a subscription. Note that a cancellation does not imply the user has churned. A user may cancel service subscription due to change of service plans or other reasons. **The criteria of "churn" is no new valid service subscription within 30 days after the current membership expires. **

The train and the test data are selected from users whose membership expire within a certain month. The train data consists of users whose subscription expires within the month of February 2017, and the test data is with users whose subscription expires within the month of March 2017. This means we are looking at user churn or renewal roughly in the month of March 2017 for train set, and the user churn or renewal roughly in the month of April 2017. Train and test sets are split by transaction date, as well as the public and private leaderboard data.

In this dataset, KKBox has included more users behaviors than the ones in train and test datasets, in order to enable participants to explore different user behaviors outside of the train and test sets. For example, a user could actively cancel the subscription, but renew within 30 days.

UPDATE: As of November 6, 2017, we have refreshed the test data to predict user churn in the month of April, 2017.

* msno: user id
* is_churn: This is the target variable. Churn is defined as whether the user did not continue the subscription within 30 days of expiration. __is_churn = 1__ means churn, __is_churn = 0__ means renewal.
* payment_method_id: payment method
* payment_plan_days: length of membership plan in days
* plan_list_price: in New Taiwan Dollar (NTD)
* actual_amount_paid: in New Taiwan Dollar (NTD)
* transaction_date: format %Y%m%d
* membership_expire_date: format %Y%m%d
* is_cancel: whether or not the user canceled the membership in this transaction.

In [1]:
import xgboost
import numpy as np
import pandas as pd

/home/data/khuangaf/miniconda2/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
transactionDf = pd.concat ([pd.read_csv('data/transactions.csv'),pd.read_csv('data/transactions_v2.csv')])
transactionDf.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,20150930,20151031,0
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,20150930,20160427,0
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,20150930,20151128,0
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,20150930,20151121,0


In [3]:
userDf = pd.concat ([pd.read_csv('data/user_logs.csv'),pd.read_csv('data/user_logs_v2.csv')])
userDf.head()

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,20150513,0,0,0,0,1,1,280.335
1,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,20150709,9,1,0,0,7,11,1658.948
2,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20150105,3,3,0,0,68,36,17364.956
3,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20150306,1,0,1,1,97,27,24667.317
4,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20150501,3,0,0,0,38,38,9649.029


In [5]:
memberDf = pd.read_csv('data/members_v3.csv')
memberDf.head()

,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,20110911
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,20110914
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,0,NaN,11,20110915
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,0,NaN,11,20110915
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,20110915


In [12]:
groupedUserDf= userDf.groupby('msno').agg('mean')

In [18]:
groupedUserDf.head()

,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
msno,,,,,,,,
+++4vcS9aMH7KWdfh5git6nA5fC5jjisd5H/NcM++WM=,2.015043e+07,1.000000,1.000000,0.000000,0.000000,0.000000,2.000000,97.411000
+++EI4HgyhgcJHIPXk/VRP7bt17+2joG39T6oEfJ+tc=,2.016042e+07,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,56.868000
+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,2.016091e+07,8.571429,1.714286,2.000000,1.000000,24.428571,25.571429,7142.395857
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,2.015677e+07,1.393519,0.401235,0.365741,0.308642,92.324074,83.540123,22812.883998
+++TipL0Kt3JvgNE9ahuJ8o+drJAnQINtxD4c5GePXI=,2.015123e+07,3.000000,3.000000,2.000000,1.000000,14.000000,22.000000,3661.527000


In [19]:
trainDf =  pd.concat([pd.read_csv('data/train.csv'), pd.read_csv('data/train_v2.csv')])
trainDf.head()

,msno,is_churn
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1


In [21]:
trainDf = trainDf.set_index('msno').join(groupedUserDf.drop('date', axis=1))
trainDf

,is_churn,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
msno,,,,,,,,
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0,5.694030,2.828358,2.679104,4.835821,21.044776,27.537313,6809.034351
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0,5.694030,2.828358,2.679104,4.835821,21.044776,27.537313,6809.034351
+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,0,2.259027,0.794349,0.682889,0.899529,25.207221,24.717425,6557.258782
+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,0,2.259027,0.794349,0.682889,0.899529,25.207221,24.717425,6557.258782
+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,0,11.049679,3.807692,2.525641,2.349359,38.850962,40.852564,10950.226471
+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,0,11.049679,3.807692,2.525641,2.349359,38.850962,40.852564,10950.226471
++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,0,4.806854,1.473520,1.211838,1.576324,17.355140,12.753894,4991.385031
++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,0,4.806854,1.473520,1.211838,1.576324,17.355140,12.753894,4991.385031
++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,0,2.482833,1.197425,0.663090,0.892704,10.585837,14.716738,3080.339884


In [29]:
transactionDf.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,20150930,20151031,0
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,20150930,20160427,0
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,20150930,20151128,0
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,20150930,20151121,0


In [ ]:
def dayBeforeExpiration(row):
    if not row['membership_expire_date'] or not row['transaction_date']: return null
    expireDay = row['membership_expire_date']%100
    transactionDay = row['transaction_date']%100
    expireMonth = (row['membership_expire_date'] % 10000) / 100
    transactionMonth = (row['transaction_date'] % 10000) / 100
    expireYear= row['membership_expire_date'] / 10000
    transactionYear= row['transaction_date']/10000
    days =  (expireYear - transactionYear)*365 +(expireMonth - transactionMonth) * 30 + expireDay - transactionDay
    if days >= 0: 
        return days
    else:
        return None

transactionDf['dayBeforeExpiration']=transactionDf.apply(dayBeforeExpiration, axis=1).head()
transactionDf.head()

In [ ]:
transactionDf.dropna(subset=['dayBeforeExpiration'])

In [4]:
pd.read_csv('data/train_v2.csv')

,msno,is_churn
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1
5,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,1
6,kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=,1
7,moRTKhKIDvb+C8ZHOgmaF4dXMLk0jOn65d7a8tQ2Eds=,1
8,dW/tPZMDh2Oz/ksduEctJbsz0MXw3kay/1AlZCq3EbI=,1
9,otEcMhAX3mU4gumUSogqgteN3oaNmhdmTkoof2iRYEE=,1
